In [2]:
import numpy as np
import os
from methods import *

In [1]:
#user inputs

#load hyperparameters
sizes = ['1_tiny', '2_small', '3_standard', '4_full']
size_folders = ['alphas_sized_decomposed_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'alphas_sized_decomposed_f1/1_tiny': [32, 32, 32, 32, 32], 
					'alphas_sized_decomposed_f1/2_small': [32, 32, 32, 32, 32],
					'alphas_sized_decomposed_f1/3_standard': [16, 16, 16, 16, 4],
					'alphas_sized_decomposed_f1/4_full': [16, 16, 16, 16, 4]}

#number of words for input
input_size_list = [50, 50, 40, 25, 25]

#alphas = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]
alphas = [0.05, 0.1, 0.2, 0.4]

a_methods = ["sr", "ri", "rd", "rs"]
#a_methods = ["rd"]
# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300


In [3]:
def run_rnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_model(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=10, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=0)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [4]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=10, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=0)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [6]:
#for each method
for a_method in a_methods:
    if not os.path.isdir('outputs_f1/'):
        os.mkdir('outputs_f1/')
    

    writer_weda = open('outputs_f1/cnn_weda_' + a_method + '_' + get_now_str() + '.txt', 'w')
    writer_eda = open('outputs_f1/cnn_eda_' + a_method + '_' + get_now_str() + '.txt', 'w')


    #for each size dataset
    for size_folder in size_folders:

        writer_weda.write(size_folder + '\n')
        writer_eda.write(size_folder + '\n')

        #get all six datasets
        dataset_folders = [size_folder + '/' + s for s in datasets]

        #for storing the performances
        performances_weda = {alpha:[] for alpha in alphas}
        performances_eda = {alpha:[] for alpha in alphas}

        #for each dataset
        for i in range(len(dataset_folders)):

            #initialize all the variables
            dataset_folder = dataset_folders[i]
            dataset = datasets[i]
            num_classes = num_classes_list[i]
            input_size = input_size_list[i]
            word2vec_pickle = dataset_folder + '/word2vec.p'
            word2vec = load_pickle(word2vec_pickle)

            #test each alpha value
            for alpha in alphas:

                train_path_weda = dataset_folder + '/train_weda_' + a_method + '_' + str(alpha) + '.txt'
                train_path_eda = dataset_folder + '/train_eda_' + a_method + '_' + str(alpha) + '.txt'

                test_path = 'alphas_sized_decomposed_f1/test/' + dataset + '/test.txt'
                
                acc = run_cnn(train_path_weda, test_path, num_classes, percent_dataset=1)
                print("cnn aug weda ", acc, alpha, a_method, dataset_folder)
                
                performances_weda[alpha].append(acc)
                
                acc = run_cnn(train_path_eda, test_path, num_classes, percent_dataset=1)
                print("cnn aug eda ", acc, alpha, a_method, dataset_folder)
                performances_eda[alpha].append(acc)
                

        writer_weda.write(str(performances_weda) + '\n')
        writer_eda.write(str(performances_eda)+"\n")
        
        print("WEDA")
        for alpha in performances_weda:
            line = str(alpha) + ' : ' + str(sum(performances_weda[alpha])/len(performances_weda[alpha]))
            writer_weda.write(line + '\n')
            print(line)
            
        print("EDA")
        for alpha in performances_eda:
            line = str(alpha) + ' : ' + str(sum(performances_eda[alpha])/len(performances_eda[alpha]))
            writer_eda.write(line + '\n')
            print(line)
  

        print(performances_weda)
        print(performances_eda)

    writer_weda.close()
    writer_eda.close()


cnn aug weda  0.7718696397941681 0.05 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7724413950829045 0.05 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7741566609491137 0.1 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7747284162378502 0.1 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7695826186392224 0.2 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.49914236706689535 0.2 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7415666094911378 0.4 sr alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7461406518010292 0.4 sr alphas_sized_decomposed_f1/1_tiny/sst2
WEDA
0.05 : 0.7718696397941681
0.1 : 0.7741566609491137
0.2 : 0.7695826186392224
0.4 : 0.7415666094911378
EDA
0.05 : 0.7724413950829045
0.1 : 0.7747284162378502
0.2 : 0.49914236706689535
0.4 : 0.7461406518010292
{0.05: [0.7718696397941681], 0.1: [0.7741566609491137], 0.2: [0.7695826186392224], 0.4: [0.7415666094911378]}
{0.05: [0.7724413950829045], 0.1: [0.7747284162

cnn aug weda  0.773013150371641 0.05 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7695826186392224 0.05 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7764436821040595 0.1 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7827329902801601 0.1 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7718696397941681 0.2 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7598627787307033 0.2 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug weda  0.7495711835334476 0.4 rd alphas_sized_decomposed_f1/1_tiny/sst2
cnn aug eda  0.7341337907375644 0.4 rd alphas_sized_decomposed_f1/1_tiny/sst2
WEDA
0.05 : 0.773013150371641
0.1 : 0.7764436821040595
0.2 : 0.7718696397941681
0.4 : 0.7495711835334476
EDA
0.05 : 0.7695826186392224
0.1 : 0.7827329902801601
0.2 : 0.7598627787307033
0.4 : 0.7341337907375644
{0.05: [0.773013150371641], 0.1: [0.7764436821040595], 0.2: [0.7718696397941681], 0.4: [0.7495711835334476]}
{0.05: [0.7695826186392224], 0.1: [0.782732990280160

In [7]:

writer_cnn = open('outputs_f1/cnn_no_aug_' + get_now_str() + '.txt', 'w')


#for each size dataset
for size_folder in size_folders:

    writer_cnn.write(size_folder + '\n')
    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]

    #for storing the performances
    performances_cnn = []

    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset_folder + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        train_path = dataset_folder + '/train_orig.txt'
        test_path = 'alphas_sized_decomposed_f1/test/' + dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn", acc)
        performances_cnn.append(acc)

        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn.append(acc)
        print("rnn", acc)
        """

    writer_cnn.write(str(performances_cnn) + '\n')

    print(performances_cnn)

writer_cnn.close()
writer_rnn.close()


cnn 0.7141223556317896
[0.7141223556317896]
cnn 0.7895940537449971
[0.7895940537449971]
cnn 0.826186392224128
[0.826186392224128]
cnn 0.8387650085763293
[0.8387650085763293]


NameError: name 'writer_rnn' is not defined